## Preprocessing

In [131]:
import pygments.lexers
import numpy as np
from gensim.models import Word2Vec

# import sys
# sys.path.append('./reentrancy')
# import reentrancy

In [132]:
# tokenize a given function
def tokenize(lexer, fn):
    tokens = list(pygments.lex(fn, lexer))
    tokenList = []
    for token in tokens:
        if tokenList and tokenList[-1] == "function":
            tokenList.append("Token.FunctionName")
        elif token[0] == pygments.token.Name.Variable or token[1].startswith("_"):
            tokenList.append("Token.Variable")
        elif token[1].startswith("\""):
            tokenList.append("Token.String")
        elif str(token[0]) != 'Token.Text.Whitespace' and str(token[0]) != 'Token.Comment.Single':
            tokenList.append(token[1])
    return tokenList

In [133]:
# return list of token lists given list of functions
def getTokenList(functionList):
    lexer = pygments.lexers.get_lexer_by_name('Solidity')
    globalList = []
    for fn in functionList:
        globalList.append(tokenize(lexer, fn))
    
    return globalList

In [134]:
# given tokenized functions (or any tokenized dataset of Solidity code), return w2v mapping
def get_w2v_mapping(tokenized_fns, token_dim):
    w2v = Word2Vec(tokenized_fns, min_count=1, size=token_dim, workers=3, window=3, sg=1)
    return w2v

In [135]:
# create set of vectorized representations of functions, where each row is a w2v token vector
def vectorize_functions(tokenized_fns, w2v_mapping):
    embedding_list = []
    for i in range(len(tokenized_fns)):
        embedding = []
        for token in tokenized_fns[i]:
            embedding.append(w2v_mapping[token])

        embedding_list.append(embedding)
    return embedding_list

In [136]:
# given list of vectorized functions, return padded version
def pad_embeddings(embeddings, max_length, token_dim):
    padded_embeddings = []
    for embedding in embeddings:
        zero_padding_cnt = max_length - len(embedding)
        pad = np.zeros((1, token_dim))
        for i in range(zero_padding_cnt):
            embedding = np.concatenate((embedding, pad), axis=0)
        padded_embeddings.append(embedding)
    return padded_embeddings

In [171]:
def preprocess(function_list, max_length, token_dim):
    tokenized_fns = getTokenList(function_list)
    print(tokenized_fns)
    print("INPUT: ", type(tokenized_fns))
    print("Tokenized Functions: ", type(tokenized_fns))
    w2v_mapping = get_w2v_mapping(tokenized_fns, token_dim)
    vectorized_fns = vectorize_functions(tokenized_fns, w2v_mapping)
    print("Vectorized Functions: ", type(vectorized_fns))
#     padded_embeddings = pad_embeddings(vectorized_fns, max_length, token_dim)
#     print("Padded Vectorizations: ", type(padded_embeddings))
    return vectorized_fns

## Model

In [172]:
import warnings
warnings.simplefilter('ignore')

import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

In [173]:
train_X_list = [
    # 0x01f8c4e3fa3edeb29e514cba738d87ce8c091d3f
    """
        function Collect(uint _am)
        public
        payable
        {
            if(balances[msg.sender]>=MinSum && balances[msg.sender]>=_am)
            {
                // <yes> <report> REENTRANCY
                if(msg.sender.call.value(_am)())
                {
                    balances[msg.sender]-=_am;
                    Log.AddMessage(msg.sender,_am,"Collect");
                }
            }
        }
    """,
    # 0x23a91059fdc9579a9fbd0edc5f2ea0bfdb70deb4
    """
        function CashOut(uint _am)
        {
            if(_am<=balances[msg.sender])
            {            
                // <yes> <report> REENTRANCY
                if(msg.sender.call.value(_am)())
                {
                    balances[msg.sender]-=_am;
                    TransferLog.AddMessage(msg.sender,_am,"CashOut");
                }
            }
        }
    """,
    # 0x627fa62ccbb1c1b04ffaecd72a53e37fc0e17839
    """
        function WithdrawToHolder(address _addr, uint _wei) 
        public
        onlyOwner
        payable
        {
            if(Holders[_addr]>0)
            {
                // <yes> <report> REENTRANCY
                if(_addr.call.value(_wei)())
                {
                    Holders[_addr]-=_wei;
                }
            }
        }
    """,
    """
        function payCharity() payable public {
          uint256 ethToPay = SafeMath.sub(totalEthCharityCollected, totalEthCharityRecieved);
          require(ethToPay > 1);
          totalEthCharityRecieved = SafeMath.add(totalEthCharityRecieved, ethToPay);
          if(!giveEthCharityAddress.call.value(ethToPay).gas(400000)()) {
             totalEthCharityRecieved = SafeMath.sub(totalEthCharityRecieved, ethToPay);
          }
        }
    """,
    """
        function withdraw()
            onlyStronghands()
            public
        {

            address _customerAddress = msg.sender;
            uint256 _dividends = myDividends(false);  


            payoutsTo_[_customerAddress] +=  (int256) (_dividends * magnitude);


            _dividends += referralBalance_[_customerAddress];
            referralBalance_[_customerAddress] = 0;


            _customerAddress.transfer(_dividends);


            onWithdraw(_customerAddress, _dividends);
        }
    """
]

train_Y = np.array([
    [1,0],
    [1,0],
    [1,0],
    [0,1],
    [0,1]
])

In [174]:
test = ["""
    function payCharity() payable public {
          uint256 ethToPay = SafeMath.sub(totalEthCharityCollected, totalEthCharityRecieved);
          require(ethToPay > 1);
          totalEthCharityRecieved = SafeMath.add(totalEthCharityRecieved, ethToPay);
          if(!giveEthCharityAddress.call.value(ethToPay).gas(400000)()) {
             totalEthCharityRecieved = SafeMath.sub(totalEthCharityRecieved, ethToPay);
          }
        }
"""]

In [175]:
max_length = 100
token_dim = 15

train_X = preprocess(train_X_list, max_length, token_dim)
train_X = np.array(train_X)
train_X.dtype

[['function', 'Token.FunctionName', 'Token.Variable', '(', 'uint', 'Token.Variable', ')', 'public', 'payable', '{', 'if', '(', 'balances', '[', 'msg.sender', ']', '>', '=', 'MinSum', '&', '&', 'balances', '[', 'msg.sender', ']', '>', '=', 'Token.Variable', ')', '{', 'if', '(', 'msg.sender', '.', 'call', '.', 'value', '(', 'Token.Variable', ')', '(', ')', ')', '{', 'balances', '[', 'msg.sender', ']', '-', '=', 'Token.Variable', ';', 'Log', '.', 'AddMessage', '(', 'msg.sender', ',', 'Token.Variable', ',', 'Token.String', ')', ';', '}', '}', '}'], ['function', 'Token.FunctionName', 'Token.Variable', '(', 'uint', 'Token.Variable', ')', '{', 'if', '(', 'Token.Variable', '<', '=', 'balances', '[', 'msg.sender', ']', ')', '{', 'if', '(', 'msg.sender', '.', 'call', '.', 'value', '(', 'Token.Variable', ')', '(', ')', ')', '{', 'balances', '[', 'msg.sender', ']', '-', '=', 'Token.Variable', ';', 'TransferLog', '.', 'AddMessage', '(', 'msg.sender', ',', 'Token.Variable', ',', 'Token.String', ')',

dtype('O')

In [176]:
train_X.shape

(5,)

In [177]:
train_X[0].shape

AttributeError: 'list' object has no attribute 'shape'

In [89]:
model = Sequential()
model.add(LSTM(128))
model.add(Dense(2, activation='softmax'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [90]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [167]:
print("Training...")
model.fit(train_X, train_Y, epochs=50)

Training...
Epoch 1/50
5/5 [==============================] - 0s 85ms/sample - loss: 0.6931 - acc: 0.6000
Epoch 2/50
5/5 [==============================] - 0s 12ms/sample - loss: 0.6912 - acc: 0.6000
Epoch 3/50
5/5 [==============================] - 0s 12ms/sample - loss: 0.6891 - acc: 0.6000
Epoch 4/50
5/5 [==============================] - 0s 12ms/sample - loss: 0.6866 - acc: 0.6000
Epoch 5/50
5/5 [==============================] - 0s 12ms/sample - loss: 0.6837 - acc: 0.6000
Epoch 6/50
5/5 [==============================] - 0s 12ms/sample - loss: 0.6804 - acc: 0.6000
Epoch 7/50
5/5 [==============================] - 0s 14ms/sample - loss: 0.6767 - acc: 0.6000
Epoch 8/50
5/5 [==============================] - 0s 13ms/sample - loss: 0.6734 - acc: 0.6000
Epoch 9/50
5/5 [==============================] - 0s 12ms/sample - loss: 0.6753 - acc: 0.6000
Epoch 10/50
5/5 [==============================] - 0s 11ms/sample - loss: 0.6755 - acc: 0.6000
Epoch 11/50
5/5 [==============================